In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt



In [8]:
pd.set_option('display.max_columns', None)


# Configure user input
train_input_file = "./Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv"
train_labels_file = "./Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv"
test_input_file = "./Pump_it_Up_Data_Mining_the_Water_Table_-_Test_set_values.csv"
numerical_preprocessing = "StandardScaler"
categorical_preprocessing = ""
model_type = ""
test_prediction_output_file = ""

In [12]:
train_values = pd.read_csv(train_input_file)
train_labels = pd.read_csv(train_labels_file)

test_values = pd.read_csv(test_input_file)

display(train_labels)


,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
...,...,...
59395,60739,functional
59396,27263,functional
59397,37057,functional
59398,31282,functional


In [20]:
# ESTO ES DE LAS SOLUCIONES DE LAB 1

# number of training samples is the number of rows of train_values
n_train_samples = len(train_values.index) 
print(f"#training samples: {n_train_samples}")
print("Are #samples and #labels in training set equal?" , n_train_samples == len(train_labels))

# number of test samples is the number of rows of test_competition_input
n_test_samples = len(test_values.index)
print(f"#test samples: {n_test_samples}")

# number of input features is the number of columns of train_values
input_features = train_values.columns
n_features = len(input_features)
print(f"\n#features: {n_features}")
print(input_features)

# the number of output classes is the number of unique status_group values in train_labels_file
output_classes = train_values["status_group"].unique()
print(f"\n#output classes: {len(output_classes)}")
print(output_classes)

# 
print("\n#training samples per output class:")
print(train_labels.value_counts())

#training samples: 59400
Are #samples and #labels in training set equal? True
#test samples: 14850

#features: 40
Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')


KeyError: 'status_group'

In [5]:
#train_values.head()
#train_values.columns

train_values.describe(include ="all")


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400.000000,59400.000000,59400,55763,59400.000000,55745,59400.000000,5.940000e+04,59398,59400.000000,59400,59029,59400,59400.000000,59400.000000,59400,59400,59400.000000,56066,59400,55522,30590,56344,59400.000000,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,NaN,NaN,356,1896,NaN,2145,NaN,NaN,37399,NaN,9,19287,21,NaN,NaN,125,2092,NaN,2,1,11,2695,2,NaN,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,NaN,NaN,2011-03-15,Government Of Tanzania,NaN,DWE,NaN,NaN,none,NaN,Lake Victoria,Madukani,Iringa,NaN,NaN,Njombe,Igosi,NaN,True,GeoData Consultants Ltd,VWC,K,True,NaN,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,NaN,NaN,572,9084,NaN,17402,NaN,NaN,3563,NaN,10248,508,5294,NaN,NaN,2503,307,NaN,51011,59400,36793,682,38852,NaN,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625
mean,37115.131768,317.650385,NaN,NaN,668.297239,NaN,34.077427,-5.706033e+00,NaN,0.474141,NaN,NaN,NaN,15.297003,5.629747,NaN,NaN,179.909983,NaN,NaN,NaN,NaN,NaN,1300.652475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21453.128371,2997.574558,NaN,NaN,693.116350,NaN,6.567432,2.946019e+00,NaN,12.236230,NaN,NaN,NaN,17.587406,9.633649,NaN,NaN,471.482176,NaN,NaN,NaN,NaN,NaN,951.620547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,NaN,NaN,-90.000000,NaN,0.000000,-1.164944e+01,NaN,0.000000,NaN,NaN,NaN,1.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,18519.750000,0.000000,NaN,NaN,0.000000,NaN,33.090347,-8.540621e+00,NaN,0.000000,NaN,NaN,NaN,5.000000,2.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37061.500000,0.000000,NaN,NaN,369.000000,NaN,34.908743,-5.021597e+00,NaN,0.000000,NaN,NaN,NaN,12.000000,3.000000,NaN,NaN,25.000000,NaN,NaN,NaN,NaN,NaN,1986.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,55656.500000,20.000000,NaN,NaN,1319.250000,NaN,37.178387,-3.326156e+00,NaN,0.000000,NaN,NaN,NaN,17.000000,5.000000,NaN,NaN,215.000000,NaN,NaN,NaN,NaN,NaN,2004.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
train_values.shape


(59400, 40)

In [16]:
# Data Exploration and Cleaning
train_values.describe(include ="all")

# Find NaN values for each of the columns
for name, values in train_values.items():
    nan_count = train_values[name].isna().sum()
    print(name + ": " + str(nan_count))


#Did not find any Nan values in numerical features, only in categorical features 
# All missing values are in categorical features, so we can handle the  
# missing values as an additional category.

id: 0
amount_tsh: 0
date_recorded: 0
funder: 3637
gps_height: 0
installer: 3655
longitude: 0
latitude: 0
wpt_name: 2
num_private: 0
basin: 0
subvillage: 371
region: 0
region_code: 0
district_code: 0
lga: 0
ward: 0
population: 0
public_meeting: 3334
recorded_by: 0
scheme_management: 3878
scheme_name: 28810
permit: 3056
construction_year: 0
extraction_type: 0
extraction_type_group: 0
extraction_type_class: 0
management: 0
management_group: 0
payment: 0
payment_type: 0
water_quality: 0
quality_group: 0
quantity: 0
quantity_group: 0
source: 0
source_type: 0
source_class: 0
waterpoint_type: 0
waterpoint_type_group: 0


In [25]:
# For each column find the possible values when categorical 
train_values.value_counts(subset="scheme_management", dropna=False)
# For numeric columns find features of the data

scheme_management
VWC                 36793
WUG                  5206
NaN                  3878
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
Name: count, dtype: int64

In [26]:
# Preprocessing

In [ ]:
# Numerical Preprocessing
if numerical_preprocessing == "StandardScaler" :

elif numerical_preprocessing == "None" :

In [ ]:
# Categorical Preprocessing
# Automatically map categorical features depending on the desired encoding type
if categorical_preprocessing == "OneHotEncoder":
    # do any of these models require me to have to drop the first column that is not encoded
    # do any of my features have more than 500 categories? if so might need to select top few 
    
elif categorical_preprocessing == "OrdinalEncoder":

elif categorical_preprocessing == "TargetEncoder":




In [ ]:

# Handle Datetime 
    # Extract features like day month year
    # Cyclical encoding using sine cosine transformation 
    # find 

In [ ]:
# visualise correlation between labels wells and the other features 